In [15]:
import re, json, pathlib, logging, time, argparse, pprint, math 
import datetime as dt
import importlib.resources as rsrc
from itertools import chain
from operator import itemgetter 
#
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import nltk as nltk
from nltk.corpus import stopwords
from whoosh.lang.porter import stem
#from googletrans import Translator
from pygoogletranslation import Translator
import circlify as crcf
import matplotlib.pyplot as plt
import random

import polmap.polmap as plmp
import postprocess.postprocess as pspr
import keyword


##MM imports
#from polmap.polmap import make_directories, preprocess_text, doc2text, SDGrefs_mapper # replaced the keyword processing block

#import fromRtoPython

from docx2python import docx2python
 
#from polmap.polmap import make_directories, preprocess_text, doc2text # replaced the keyword processing block
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [29]:
results_df = pd.read_excel('output/SDGS_61_Green_Deal_Test/output/6-results/mapping_.xlsx', sheet_name='Summary', index_col=0)
pp_def = pd.read_excel('keywords/goal_target_list.xlsx')
results_df["Target"]=results_df["Target"].astype(str)
pp_def["Target"]=pp_def["Target"].astype(str)
results_df=pd.merge(results_df, pp_def, on='Target', how='left')
results_df=results_df.drop(['Goal_y','goal_id','Unnamed: 0','tar_ID'],axis=1)
results_df=results_df.rename(columns={"Goal_x": "Goal"})
results_df.head()

,Goal,Target,Sum_of_keys,Count_of_keys,list_of_keys,target_description,goal_description,goal_color,MAIN_priority,SEC_priority
0,SDG 1,1.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,SDG 1,1.1,0,0,NaN,"By 2030, eradicate extreme poverty for all people everywhere, currently measured as people living on less than $1.25 a day",No poverty,#e5243b,NaN,Human Development
2,SDG 1,1.2,0,0,NaN,"By 2030, reduce at least by half the proportion of men, women and children of all ages living in poverty in all its dimensions according to national definitions",No poverty,#e5243b,NaN,Human Development
3,SDG 1,1.3,0,0,NaN,"Implement nationally appropriate social protection systems and measures for all, including floors, and by 2030 achieve substantial coverage of the poor and the vulnerable",No poverty,#e5243b,Human Development,NaN
4,SDG 1,1.4,0,0,NaN,"By 2030, ensure that all men and women, in particular the poor and the vulnerable, have equal rights to economic resources, as well as access to basic services, ownership and control over land and other forms of property, inheritance, natural resources, appropriate new technology and financial services, including microfinance",No poverty,#e5243b,NaN,Human Development


In [48]:
main_df=results_df.groupby(by=['MAIN_priority','Goal']).sum().reset_index()
main_df.rename(columns={'MAIN_priority': 'priority'}, inplace=True)
sec_df=results_df.groupby(by=['SEC_priority','Goal']).sum().reset_index()
sec_df.rename(columns={'SEC_priority': 'priority'}, inplace=True)
priorities_df = main_df.append(sec_df, ignore_index=True)
priorities_df = priorities_df.groupby(by=['priority','Goal']).sum().reset_index()
priorities_df.head(100)

,priority,Goal,Sum_of_keys,Count_of_keys
0,Digitalisation,SDG 12,0,0
1,Digitalisation,SDG 14,0,0
2,Digitalisation,SDG 17,0,0
3,Digitalisation,SDG 2,0,0
4,Digitalisation,SDG 4,1,1
5,Digitalisation,SDG 5,0,0
6,Digitalisation,SDG 8,0,0
7,Digitalisation,SDG 9,10,4
8,Governance,SDG 10,1,1
9,Governance,SDG 11,3,3


In [62]:
polpri_bubbledict={"name": "sdgs", "children": None}
polpri_ls=[]

polpri=priorities_df.groupby(by=["priority"]).sum()

for polpri, count in zip(polpri.index.tolist(), polpri['Sum_of_keys']):
    polpri_dict={}
    if count > 0:
        polpri_dict["name"]=polpri
        polpri_dict["size"]=count
        polpri_dict["children"] = None
        polpri_ls.append(polpri_dict)
    else:
        continue

for polpri in polpri_ls:
    goal_ls=[]

    if polpri["size"] >0 :
        print(polpri["size"])
        goaldf = priorities_df[priorities_df.priority == polpri["name"]]

        for goal, count in zip(goaldf['Goal'], goaldf['Sum_of_keys']):
            goal_dict={}
            if count > 0:
                goal_dict["name"]=goal
                goal_dict["size"]=count
                goal_dict["children"] = None
                goal_ls.append(goal_dict)                    
            else:
                continue

    polpri['children']=goal_ls

polpri_bubbledict['children']=polpri_ls

with open('polpri_bubbleplot.json',mode='w', encoding='utf-8') as f:
    json.dump(polpri_bubbledict, f)
    


11
21
167
45
20


In [ ]:
df = pd.read_excel('output/The_EU_Green_Deal_/output/6-results/results_.xlsx', sheet_name='priorities', index_col=0)

pp_colors={'Human Development':'#FECA4D',
'Growth Jobs':'#FABE13',
'Green Deal':'#73936F',
'Partnership':'#1A6835',
'Digital':'#72ABDC',
'Governance':'#E2809B',
'Digitalisation':'#4D9CD5',
'Migration':'#DE6189'
}

df.rename(columns = {'priority':'name', 'Count':'size'}, inplace = True)
df['goal_color']=df['name'].map(pp_colors)
df.to_dict('index')


In [ ]:
input = 'output/results_green_deal_test.xlsx'

result_df = pd.ExcelFile(input)

print(result_df.sheet_names)

targets, goals, total = result_df.parse('target_count'), result_df.parse('goal_overview'), result_df.parse('total_count_(goals_+_targets)')

# print(result_df.sheet_names)

In [ ]:
targets.rename(columns={'Count':'Target_Count', 'Num_keys':'Tgt_nr.keys'}, inplace=True) 
targets.groupby(['Policy','Goal','Target']).sum()
dat_target_join_keys = raw_df.groupby(['Policy', 'Target'])["Keyword"].apply(lambda x : " - ".join(x.astype(str))).reset_index()

In [ ]:
targets.head

In [ ]:
goals.rename(columns = {'Count':'Goal_Count', 'Goal_nr.keys'}, inplace = True) 
goals.groupby(['Policy','Goal']).sum()